<a href="https://colab.research.google.com/github/Tom-Jung/legendary-funicular/blob/main/Ch8_1.%20ICIO_Indicators_0915.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np   # Numpy import
import pandas as pd  # Pandas import

In [3]:
np.set_printoptions(precision=3,suppress=True) # 결과를 소수 3자리까지만 보여주는 옵션

In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

In [5]:
# icio =pd.read_csv('/content/drive/MyDrive/ICIO/ICIO2021_2018.csv')

In [6]:
url_icio = "https://raw.githubusercontent.com/Tom-Jung/legendary-funicular/main/ICIO_4cons_3secs_1Fd.csv"  # 4국 9부문 2020년 ICIO 불러오기
icio = pd.read_csv(url_icio)
icio.head()

,Unnamed: 0,Unnamed: 1,한국,한국.1,한국.2,중국,중국.1,중국.2,미국,미국.1,미국.2,ROW,ROW.1,ROW.2,한국.3,중국.3,미국.3,ROW.3,총계
0,국가,산업,농산품,공산품,서비스,농산품,공산품,서비스,농산품,공산품,서비스,농산품,공산품,서비스,최종수요,최종수요,최종수요,최종수요,총수요
1,한국,농산품,2.5,28.2,10.2,0,0,0,0,0.2,0.1,0.1,0.2,0.1,15.2,0.1,0.1,0.2,57.2
2,한국,공산품,11,559.6,259.3,5,124.6,29.5,0.4,19.3,13.6,6.1,130.1,60.2,245.8,39.1,35.5,129.5,1668.5
3,한국,서비스,8.5,263.3,542.1,1.1,13.9,9.9,0,1.7,4.2,0.8,14.4,27,1161.1,12.8,8.2,28.6,2097.7
4,중국,농산품,0,0.4,0.1,438.9,969.1,150,0.3,0.9,0.3,1.2,5,1.3,0.3,550.9,0.5,6.7,2125.9


In [7]:
icio_1 = icio.to_numpy()
icio_2=icio_1[1:,2:].astype(float)
icio_2

array([[     2.5,     28.2,     10.2,      0. ,      0. ,      0. ,
             0. ,      0.2,      0.1,      0.1,      0.2,      0.1,
            15.2,      0.1,      0.1,      0.2,     57.2],
       [    11. ,    559.6,    259.3,      5. ,    124.6,     29.5,
             0.4,     19.3,     13.6,      6.1,    130.1,     60.2,
           245.8,     39.1,     35.5,    129.5,   1668.5],
       [     8.5,    263.3,    542.1,      1.1,     13.9,      9.9,
             0. ,      1.7,      4.2,      0.8,     14.4,     27. ,
          1161.1,     12.8,      8.2,     28.6,   2097.7],
       [     0. ,      0.4,      0.1,    438.9,    969.1,    150. ,
             0.3,      0.9,      0.3,      1.2,      5. ,      1.3,
             0.3,    550.9,      0.5,      6.7,   2125.9],
       [     0.7,     53.2,     18.3,    457.1,   8094.6,   3401.8,
             1.8,     88.1,     94.8,     18.1,    478. ,    297.8,
            23.3,   3220.6,    237. ,    699.6,  17184.8],
       [     0.1,      6.

### 한국 1, 중국2, 미국3, ROW4

## TiVa indicators guide

[TiVA2022_Indicators_Guide](TiVA2022_Indicators_Guide.pdf)

In [8]:
PROD = icio_2[-1,:12].astype(float) # 산출액
VALU = icio_2[-2, 0:12].astype(float) # 부가가치
PROD_VASH = VALU/PROD
PROD_VASH

array([0.526, 0.277, 0.518, 0.442, 0.225, 0.508, 0.407, 0.384, 0.596,
       0.592, 0.344, 0.573])

In [9]:
ct_m = np.tile(PROD, reps=[12,1]) # 산출액을 12번 행반복
nae = icio_2[0:12,0:12]   # 거래표

Ad =(nae/ct_m)  # 투입계수

In [10]:
Ad_df = pd.DataFrame(Ad, columns = [['한국', '한국', '한국', '중국', '중국', '중국','미국', '미국', '미국','ROW', 'ROW', 'ROW'],
                                    ['농산품', '공산품', '서비스', '농산품', '공산품', '서비스','농산품', '공산품', '서비스','농산품', '공산품', '서비스']],

                                    index=[['한국', '한국', '한국', '중국', '중국', '중국','미국', '미국', '미국','ROW', 'ROW', 'ROW'],
                                           ['농산품', '공산품', '서비스', '농산품', '공산품', '서비스','농산품', '공산품', '서비스','농산품', '공산품', '서비스']])
Ad_df.index.names=['국가', '상품']
# Ad_df

## 생산유발계수(Lf)

In [11]:
Lf = np.linalg.inv(np.identity(12)-Ad)    # 생산유발계수
Lf_sm_cl = np.hstack((Lf, Lf.sum(axis=1).reshape([12,1]))) # 생산유발계수 열합
Lf_sm_rw = np.vstack((Lf_sm_cl,Lf_sm_cl.sum(axis=0))) # 생산유발계수 행합
Lf

array([[1.054, 0.03 , 0.012, 0.   , 0.001, 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   ],
       [0.361, 1.582, 0.267, 0.015, 0.029, 0.012, 0.004, 0.009, 0.002,
        0.006, 0.014, 0.005],
       [0.288, 0.343, 1.408, 0.005, 0.009, 0.005, 0.001, 0.003, 0.001,
        0.002, 0.005, 0.002],
       [0.005, 0.011, 0.004, 1.312, 0.158, 0.06 , 0.003, 0.004, 0.001,
        0.003, 0.006, 0.002],
       [0.068, 0.136, 0.056, 0.67 , 2.145, 0.608, 0.027, 0.052, 0.016,
        0.026, 0.07 , 0.027],
       [0.02 , 0.039, 0.021, 0.322, 0.443, 1.414, 0.008, 0.014, 0.005,
        0.008, 0.02 , 0.009],
       [0.004, 0.003, 0.001, 0.002, 0.002, 0.001, 1.227, 0.051, 0.005,
        0.002, 0.003, 0.001],
       [0.023, 0.039, 0.018, 0.009, 0.016, 0.007, 0.269, 1.367, 0.103,
        0.013, 0.031, 0.011],
       [0.025, 0.035, 0.022, 0.01 , 0.014, 0.008, 0.477, 0.459, 1.509,
        0.015, 0.031, 0.02 ],
       [0.014, 0.022, 0.009, 0.017, 0.016, 0.007, 0.028, 0.017, 0.003,
        1.167, 0.087

부가가치유발계수(Lv)

In [12]:
Va_r = icio_2[-2,:12]/PROD # 부가가치율
Va_r

array([0.526, 0.277, 0.518, 0.442, 0.225, 0.508, 0.407, 0.384, 0.596,
       0.592, 0.344, 0.573])

In [13]:
Va_m = np.diag(Va_r)  # 부가가치율 대각행렬

In [14]:
Lv = Va_m@Lf  #부가가치유발계수
Lv
Lv_sm_cl = np.hstack((Lv, Lv.sum(axis=1).reshape([12,1]))) # 부가가치유발계수 열합
Lv_sm_rw = np.vstack((Lv_sm_cl,Lv_sm_cl.sum(axis=0))) # 부가가치유발계수 행합
Lv_sm_rw

array([[ 0.554,  0.016,  0.006,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  0.   ,  0.577],
       [ 0.1  ,  0.437,  0.074,  0.004,  0.008,  0.003,  0.001,  0.002,
         0.001,  0.002,  0.004,  0.001,  0.638],
       [ 0.149,  0.178,  0.729,  0.003,  0.005,  0.002,  0.001,  0.001,
         0.   ,  0.001,  0.002,  0.001,  1.073],
       [ 0.002,  0.005,  0.002,  0.58 ,  0.07 ,  0.027,  0.001,  0.002,
         0.001,  0.001,  0.002,  0.001,  0.694],
       [ 0.015,  0.031,  0.013,  0.151,  0.484,  0.137,  0.006,  0.012,
         0.004,  0.006,  0.016,  0.006,  0.879],
       [ 0.01 ,  0.02 ,  0.01 ,  0.163,  0.225,  0.718,  0.004,  0.007,
         0.003,  0.004,  0.01 ,  0.005,  1.179],
       [ 0.002,  0.001,  0.001,  0.001,  0.001,  0.   ,  0.499,  0.021,
         0.002,  0.001,  0.001,  0.   ,  0.53 ],
       [ 0.009,  0.015,  0.007,  0.003,  0.006,  0.003,  0.103,  0.525,
         0.04 ,  0.005,  0.012,  0.004,  0.732],
       [ 0.015,  0.021,  0.013, 

In [15]:
Lf_df = pd.DataFrame(Lf.round(3), columns = [['한국', '한국', '한국', '중국', '중국', '중국','미국', '미국', '미국','ROW', 'ROW', 'ROW'],
                                    ['농산품', '공산품', '서비스', '농산품', '공산품', '서비스','농산품', '공산품', '서비스','농산품', '공산품', '서비스']],

                                    index=[['한국', '한국', '한국', '중국', '중국', '중국','미국', '미국', '미국','ROW', 'ROW', 'ROW'],
                                           ['농산품', '공산품', '서비스', '농산품', '공산품', '서비스','농산품', '공산품', '서비스','농산품', '공산품', '서비스']])
Lf_df.index.names=['국가', '상품']
Lf_df

한국                   중국                   미국                  ROW  \
           농산품    공산품    서비스    농산품    공산품    서비스    농산품    공산품    서비스    농산품   
국가  상품                                                                          
한국  농산품  1.054  0.030  0.012  0.000  0.001  0.000  0.000  0.000  0.000  0.000   
    공산품  0.361  1.582  0.267  0.015  0.029  0.012  0.004  0.009  0.002  0.006   
    서비스  0.288  0.343  1.408  0.005  0.009  0.005  0.001  0.003  0.001  0.002   
중국  농산품  0.005  0.011  0.004  1.312  0.158  0.060  0.003  0.004  0.001  0.003   
    공산품  0.068  0.136  0.056  0.670  2.145  0.608  0.027  0.052  0.016  0.026   
    서비스  0.020  0.039  0.021  0.322  0.443  1.414  0.008  0.014  0.005  0.008   
미국  농산품  0.004  0.003  0.001  0.002  0.002  0.001  1.227  0.051  0.005  0.002   
    공산품  0.023  0.039  0.018  0.009  0.016  0.007  0.269  1.367  0.103  0.013   
    서비스  0.025  0.035  0.022  0.010  0.014  0.008  0.477  0.459  1.509  0.015   
ROW 농산품  0.014  0.022  0.009  0.017  0.016  0.007  0.028  0.017  0.003  1.167   
    공산품  0.148  0.337  0.132  0.092  0.184  0.082  0.089  0.182  0.041  0.247   
    서비스  0.084  0.158  0.087  0.055  0.091  0.049  0.058  0.092  0.033  0.290   

                       
           공산품    서비스  
국가  상품                 
한국  농산품  0.000  0.000  
    공산품  0.014  0.005  
    서비스  0.005  0.002  
중국  농산품  0.006  0.002  
    공산품  0.070  0.027  
    서비스  0.020  0.009  
미국  농산품  0.003  0.001  
    공산품  0.031  0.011  
    서비스  0.031  0.020  
ROW 농산품  0.087  0.020  
    공산품  1.591  0.208  
    서비스  0.487  1.483

In [16]:
Lv_df = pd.DataFrame(Lv.round(3), columns = [['한국', '한국', '한국', '중국', '중국', '중국','미국', '미국', '미국','ROW', 'ROW', 'ROW'],
                                    ['농산품', '공산품', '서비스', '농산품', '공산품', '서비스','농산품', '공산품', '서비스','농산품', '공산품', '서비스']],

                                    index=[['한국', '한국', '한국', '중국', '중국', '중국','미국', '미국', '미국','ROW', 'ROW', 'ROW'],
                                           ['농산품', '공산품', '서비스', '농산품', '공산품', '서비스','농산품', '공산품', '서비스','농산품', '공산품', '서비스']])
Lv_df.index.names=['국가', '상품']
Lv_df

한국                   중국                   미국                  ROW  \
           농산품    공산품    서비스    농산품    공산품    서비스    농산품    공산품    서비스    농산품   
국가  상품                                                                          
한국  농산품  0.554  0.016  0.006  0.000  0.000  0.000  0.000  0.000  0.000  0.000   
    공산품  0.100  0.437  0.074  0.004  0.008  0.003  0.001  0.002  0.001  0.002   
    서비스  0.149  0.178  0.729  0.003  0.005  0.002  0.001  0.001  0.000  0.001   
중국  농산품  0.002  0.005  0.002  0.580  0.070  0.027  0.001  0.002  0.001  0.001   
    공산품  0.015  0.031  0.013  0.151  0.484  0.137  0.006  0.012  0.004  0.006   
    서비스  0.010  0.020  0.010  0.163  0.225  0.718  0.004  0.007  0.003  0.004   
미국  농산품  0.002  0.001  0.001  0.001  0.001  0.000  0.499  0.021  0.002  0.001   
    공산품  0.009  0.015  0.007  0.003  0.006  0.003  0.103  0.525  0.040  0.005   
    서비스  0.015  0.021  0.013  0.006  0.009  0.005  0.284  0.274  0.900  0.009   
ROW 농산품  0.008  0.013  0.005  0.010  0.010  0.004  0.016  0.010  0.002  0.690   
    공산품  0.051  0.116  0.045  0.032  0.063  0.028  0.030  0.062  0.014  0.085   
    서비스  0.048  0.091  0.050  0.031  0.052  0.028  0.033  0.052  0.019  0.166   

                       
           공산품    서비스  
국가  상품                 
한국  농산품  0.000  0.000  
    공산품  0.004  0.001  
    서비스  0.002  0.001  
중국  농산품  0.002  0.001  
    공산품  0.016  0.006  
    서비스  0.010  0.005  
미국  농산품  0.001  0.000  
    공산품  0.012  0.004  
    서비스  0.019  0.012  
ROW 농산품  0.051  0.012  
    공산품  0.547  0.071  
    서비스  0.279  0.850

In [17]:
Lf_df.to_csv('생산유발계수.csv', encoding='utf_8-sig')
Lv_df.to_csv('부가가치유발계수.csv', encoding='utf_8-sig')

## DVA(Domestic Value-added) and FVA(Foreign Value-added)

In [21]:
Fd = icio_2[0:12,12:16]
Fd

array([[   15.2,     0.1,     0.1,     0.2],
       [  245.8,    39.1,    35.5,   129.5],
       [ 1161.1,    12.8,     8.2,    28.6],
       [    0.3,   550.9,     0.5,     6.7],
       [   23.3,  3220.6,   237. ,   699.6],
       [   12.2,  8961.9,    39.8,   145.6],
       [    0.7,     2.7,   102.2,    16.3],
       [   11.5,    32.7,  2574.7,   396.8],
       [   16.9,    61. , 16683.7,   420.9],
       [    1.6,    17.5,    13.6,  1637.3],
       [   54.5,   215.1,   726.5,  9469.7],
       [   37.3,   280. ,   408.9, 34204.8]])

In [24]:
DFVA = Lv@Fd
DFVA

array([[   19.55 ,     2.987,     1.638,     5.926],
       [  195.378,    78.169,    38.427,   149.563],
       [  892.933,    55.965,    27.228,   110.219],
       [    5.792,   784.011,    34.619,   115.482],
       [   36.613,  2876.407,   227.792,   734.207],
       [   32.04 ,  7252.814,   153.334,   536.887],
       [    1.752,     7.903,   141.071,    39.174],
       [   19.425,    70.685,  2035.997,   498.   ],
       [   41.127,   148.762, 15764.878,  1094.128],
       [   14.302,    99.854,   114.824,  2035.764],
       [  117.106,   619.473,   847.125,  7850.916],
       [  130.295,   744.567,  1027.725, 32068.446]])

In [26]:
DFVA_df = pd.DataFrame(DFVA, columns = ['한국', '중국', '미국', 'ROW'],

                                    index=[['한국', '한국', '한국', '중국', '중국', '중국','미국', '미국', '미국','ROW', 'ROW', 'ROW'],
                                           ['농산품', '공산품', '서비스', '농산품', '공산품', '서비스','농산품', '공산품', '서비스','농산품', '공산품', '서비스']])
DFVA_df.index.names=['국가', '상품']
DFVA_df

한국           중국            미국           ROW
국가  상품                                                      
한국  농산품   19.549819     2.986852      1.638402      5.925908
    공산품  195.378371    78.168595     38.427067    149.563485
    서비스  892.933404    55.964729     27.228129    110.219365
중국  농산품    5.791782   784.011442     34.618737    115.481518
    공산품   36.613019  2876.406795    227.792370    734.206863
    서비스   32.039840  7252.814238    153.334385    536.886597
미국  농산품    1.752337     7.903081    141.071412     39.173635
    공산품   19.424703    70.685424   2035.996991    498.000295
    서비스   41.127384   148.761518  15764.878435   1094.127894
ROW 농산품   14.301901    99.854305    114.823588   2035.764022
    공산품  117.105517   619.472971    847.125111   7850.915608
    서비스  130.294978   744.566892   1027.725492  32068.445658

In [32]:
DFVA_df.to_csv('DFVA.csv', encoding="utf-8-sig")

In [ ]:
import scipy.linalg as linalg

In [ ]:
A = icio_2[:12,:12].astype(float)
A

array([[    2.5,    28.2,    10.2,     0. ,     0. ,     0. ,     0. ,
            0.2,     0.1,     0.1,     0.2,     0.1],
       [   11. ,   559.6,   259.3,     5. ,   124.6,    29.5,     0.4,
           19.3,    13.6,     6.1,   130.1,    60.2],
       [    8.5,   263.3,   542.1,     1.1,    13.9,     9.9,     0. ,
            1.7,     4.2,     0.8,    14.4,    27. ],
       [    0. ,     0.4,     0.1,   438.9,   969.1,   150. ,     0.3,
            0.9,     0.3,     1.2,     5. ,     1.3],
       [    0.7,    53.2,    18.3,   457.1,  8094.6,  3401.8,     1.8,
           88.1,    94.8,    18.1,   478. ,   297.8],
       [    0.1,     6. ,     8.4,   240. ,  2611. ,  3490.9,     0.2,
            8.6,    18.3,     2.7,    55.3,   104.2],
       [    0.1,     1.3,     0.4,     2. ,     4.7,     0.8,    83. ,
          208.8,    20. ,     4.3,    16.4,     2.9],
       [    0.4,    21.7,     9.3,     3. ,    57.4,    16.8,    67.7,
         1665.8,  1454.8,    14.8,   335.6,   174. ],


In [ ]:
FD = icio_2[0:12,12:16]
FD

array([[   15.2,     0.1,     0.1,     0.2],
       [  245.8,    39.1,    35.5,   129.5],
       [ 1161.1,    12.8,     8.2,    28.6],
       [    0.3,   550.9,     0.5,     6.7],
       [   23.3,  3220.6,   237. ,   699.6],
       [   12.2,  8961.9,    39.8,   145.6],
       [    0.7,     2.7,   102.2,    16.3],
       [   11.5,    32.7,  2574.7,   396.8],
       [   16.9,    61. , 16683.7,   420.9],
       [    1.6,    17.5,    13.6,  1637.3],
       [   54.5,   215.1,   726.5,  9469.7],
       [   37.3,   280. ,   408.9, 34204.8]])

In [ ]:
mtx_for_int= linalg.block_diag(np.ones((3,3)),np.ones((3,3)),np.ones((3,3)),np.ones((3,3)))
mtx_for_tra=np.where(mtx_for_int == 1, 0, 1)

mtx_for_fnl= linalg.block_diag(np.ones((3,1)),np.ones((3,1)),np.ones((3,1)),np.ones((3,1)))
mtx_for_fnl_tra=np.where(mtx_for_fnl == 1, 0, 1)

INT = icio_2[0:12,0:12].astype(float)

EXIM_INT_mt = (INT*mtx_for_tra).reshape([12,12])  # 중간재 수출입
EXGR_INT = EXIM_INT_mt.sum(axis=1).reshape([12,1])

FD = icio_2[0:12,12:16].astype(float)

EXIM_FNL_mt = (FD*mtx_for_fnl_tra).reshape([12,4])  # 최종재 수출입
EXGR_FNL = EXIM_FNL_mt.sum(axis=1).reshape([12,1])

# EXIM = EXIM_INT_mt + EXIM_FNL_mt  # 총수출입

# Ex_all = np.concatenate((EXGR, EXGR_INT, EXGR_FNL),axis=1)


EXIM_INT_df = pd.DataFrame(EXIM_INT_mt, columns = [['한국','한국', '한국','중국','중국','중국','미국','미국','미국','ROW','ROW','ROW'],
                          ['농산품','공산품', '서비스','농산품','공산품', '서비스','농산품','공산품', '서비스','농산품','공산품', '서비스']],

                         index=[['한국','한국', '한국','중국','중국','중국','미국','미국','미국','ROW','ROW','ROW'],
                          ['농산품','공산품', '서비스','농산품','공산품', '서비스','농산품','공산품', '서비스','농산품','공산품', '서비스']])

# EXIM_FNL_df = pd.DataFrame(EXIM_FNL_mt, columns = [['한국','중국', '미국','ROW'],

#                          index=[['한국','한국', '한국','중국','중국','중국','ROW','ROW','ROW'],
#                           ['농산품','공산품', '서비스','농산품','공산품', '서비스','농산품','공산품', '서비스']])
EXIM_INT_df
FD

# Ex_all_df = pd.DataFrame(Ex_all, columns = ['총수출', '중간재수출', '최종재수출'],
#                          index=[['한국','한국', '한국','중국','중국','중국','ROW','ROW','ROW'],
#                           ['농산품','공산품', '서비스','농산품','공산품', '서비스','농산품','공산품', '서비스']])
# Ex_all_df
# print('중간재 수출 : \n', EXGR_INT, '\n')
# print('최종재 수출 : \n', EXGR_FNL, '\n')
# print('총수출 : \n', EXGR, '\n')

array([[   15.2,     0.1,     0.1,     0.2],
       [  245.8,    39.1,    35.5,   129.5],
       [ 1161.1,    12.8,     8.2,    28.6],
       [    0.3,   550.9,     0.5,     6.7],
       [   23.3,  3220.6,   237. ,   699.6],
       [   12.2,  8961.9,    39.8,   145.6],
       [    0.7,     2.7,   102.2,    16.3],
       [   11.5,    32.7,  2574.7,   396.8],
       [   16.9,    61. , 16683.7,   420.9],
       [    1.6,    17.5,    13.6,  1637.3],
       [   54.5,   215.1,   726.5,  9469.7],
       [   37.3,   280. ,   408.9, 34204.8]])

In [ ]:
aa=EXIM_FNL_mt.sum(axis=1)
aa

array([  0.4, 204.1,  49.6,   7.5, 959.9, 197.6,  19.7, 441. , 498.8,
        32.7, 996.1, 726.2])

In [ ]:
EXIM_FNL_mt.shape

(12, 4)

In [ ]:
# 국가별 제품별 수출입
# 중간재
EXGR_INT = EXIM_INT_mt.sum(axis=1).reshape([12,1]) # 중간재 수출
IMGR_INT = EXIM_INT_mt.sum(axis=0).reshape([12,1]) # 중간재 수입

EXGR_FNL = EXIM_FNL_mt.sum(axis=1).reshape([12,1]) # 최종재 수출
IMGR_FNL = EXIM_FNL_mt.sum(axis=0).reshape([4,1]) # 최종재 수입

EXGR = EXIM.sum(axis=1).reshape([12,1])
IMGR = EXIM.sum(axis=0).reshape([12,1])

EXGR_IMGR_vec = np.hstack((EXGR_INT,EXGR_FNL,IMGR_INT,IMGR_FNL, EXGR, IMGR))
EXGR_IMGR_vec

EXGR_IMGR_vec_df = pd.DataFrame(EXGR_IMGR_vec, columns = ['EXGR_INT','EXGR_FNL', 'IMGR_INT','IMGR_FNL','EXGR','IMGR'],

                         index=[['한국','한국', '한국','중국','중국','중국','미국','미국','미국','ROW','ROW','ROW'],
                          ['농산품','공산품', '서비스','농산품','공산품', '서비스','농산품','공산품', '서비스','농산품','공산품', '서비스']])
EXGR_IMGR_vec_df

NameError: ignored

In [ ]:
# 경상수지 및 파트너 비중(산업을 나눌 필요가 없으므로 Trade_matrix를 국가별,전산업으로 변환)
Mt_cd_all = np.array([[1,1,1,0,0,0,0,0,0],
                     [0,0,0,1,1,1,0,0,0],
                     [0,0,0,0,0,0,1,1,1]])
trade_1sec = (Mt_cd_all@EXIM)@Mt_cd_all.T
trade_1sec  # trade 매트릭스
exp_1sec = trade_1sec.sum(axis=1).reshape([1,3])
imp_1sec = trade_1sec.sum(axis=0).reshape([3,1])
BALGR = (exp_1sec.T - imp_1sec).reshape([1,3]) #경상수지(Gross trade balance)

BALGR_df = pd.DataFrame(BALGR, columns = ['한국','중국', 'ROW'],

                         index=['BALGR'])

exp_1sec_con = trade_1sec.reshape([3,3])/np.tile((exp_1sec.reshape([1,3]))[1,3])  #전산업, 국가별 수출 매트릭스

exp_1sec_con

In [ ]:
aa=np.tile(exp_1sec[1,3])

IndexError: ignored

In [ ]:
exp_1sec

array([[ 5818., 24630., 24898.]])

In [ ]:
exp_1sec.shape

(1, 3)

In [ ]:

# IMGR_INT_mt = (INT*mtx_for_tra).reshape([9,9])  # 중간재 수입
# IMGR_INT = IMGR_INT_mt.sum(axis=0).reshape([1,9])

# IMGR_FNL_mt = (FD*mtx_for_tra).reshape([9,9])  # 최종재 수입
# IMGR_FNL = IMGR_FNL_mt.sum(axis=0).reshape([1,9])

# IMGR = IMGR_INT + IMGR_FNL

# print(IMGR_INT_mt)

# Im_all = np.concatenate((IMGR, IMGR_INT, IMGR_FNL),axis=0).T

# Im_all_df = pd.DataFrame(Im_all, columns = ['총수입', '중간재수입', '최종재수입'],
#                          index=[['한국','한국', '한국','중국','중국','중국','ROW','ROW','ROW'],
#                           ['농산품','공산품', '서비스','농산품','공산품', '서비스','농산품','공산품', '서비스']])
# Im_all_df

# # print('중간재 수입 : \n', IMGR_INT.T, '\n')
# # print('최종재 수입 : \n', IMGR_FNL.T, '\n')
# # print('총수입 : \n', IMGR.T, '\n')

In [ ]:
Mtx = np.zeros((18,3195)) # 부문통합을 위한 코드가 많을 경우 상기의 코드를 수작업으로 미리 하는 것이 쉽자 않으므로 반복문을 통해 3x33 임시코드 만들기
cd_3 = cd_for_itr[:,1]
cd_33 = cd_for_itr[:,0]
row=cd_3
col=cd_33
for i, j in zip(row,col):
  Mtx[i-1,j-1] = 1
print(Mtx)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [1. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
code=Mtx
code.shape

(18, 3195)

In [ ]:
nae_tot_row = code@icio_2[0:3195,0:3195] # 중간투입(수요)의 행을 18부문으로 만들기
nae_tot_row.shape

ValueError: ignored

In [ ]:
nae_tot_col = nae_tot_row@code.T  # 수작업한 code 이용
nae_tot_col.sum()

In [ ]:
url_cdtx = "https://raw.githubusercontent.com/Tom-Jung/legendary-funicular/main/cd_tax.csv"  #코드매치 파일 불러오기
cd_tx = pd.read_csv(url_cdtx)
cd_tx.shape

In [ ]:
url_cns = "https://raw.githubusercontent.com/Tom-Jung/legendary-funicular/main/cd_cons.csv"  #코드매치 파일 불러오기
cd_cns = pd.read_csv(url_cns)
cd_cns.shape

In [ ]:
# Mtx_tx= np.zeros((6,3195)) # 부문통합을 위한 코드가 많을 경우 상기의 코드를 수작업으로 미리 하는 것이 쉽자 않으므로 반복문을 통해 3x33 임시코드 만들기
# cd_3 = cd_for_itr_tx[:,1]
# cd_33 = cd_for_itr_tx[:,0]
# row=cd_3
# col=cd_33
# for i, j in zip(row,col):
#   Mtx_tx[i-1,j-1] = 1
# print(Mtx_tx)

In [ ]:
ccdd_cns = cd_cns.to_numpy()
cd_for_itr_cns = ccdd_cns[:,1:]
# cd_for_itr_tx

In [ ]:
Mtx_cns= np.zeros((6,67)) # 부문통합을 위한 코드가 많을 경우 상기의 코드를 수작업으로 미리 하는 것이 쉽자 않으므로 반복문을 통해 3x33 임시코드 만들기
cd_3 = cd_for_itr_cns[:,1]
cd_33 = cd_for_itr_cns[:,0]
row=cd_3
col=cd_33
for i, j in zip(row,col):
  Mtx_cns[i-1,j-1] = 1
print(Mtx_cns)

In [ ]:
# code_tx = Mtx_tx
code_cns = Mtx_cns
code_cns.shape

In [ ]:
nae_tx = code_cns@icio_2[3195:3262,0:3195]@code.T # 세금 행을 6부문으로 만들기
nae_tx

In [ ]:
va_row = icio_2[3262:,0:3195]@code.T # 중투합, 피용자보수, 영업잉여, 고정자본소모, 기타생산세, 부가가치계, 산출액(7X3)
va_row

In [ ]:
url_cdfd = "https://raw.githubusercontent.com/Tom-Jung/legendary-funicular/main/cd_fd.csv"  #코드매치 파일 불러오기
cd_fd = pd.read_csv(url_cdfd)
cd_fd.head()

In [ ]:
ccdd_fd = cd_fd.to_numpy()
cd_for_itr_fd = ccdd_fd[:,1:]
cd_for_itr_fd.shape

In [ ]:
Mtx_fd= np.zeros((18,402)) # 부문통합을 위한 코드가 많을 경우 상기의 코드를 수작업으로 미리 하는 것이 쉽자 않으므로 반복문을 통해 3x33 임시코드 만들기
cd_3 = cd_for_itr_fd[:,1]
cd_33 = cd_for_itr_fd[:,0]
row=cd_3
col=cd_33
for i, j in zip(row,col):
  Mtx_fd[i-1,j-1] = 1
print(Mtx_fd)

In [ ]:
code_fd = Mtx_fd
code_fd.shape

In [ ]:
fd = code@icio_2[0:3195,3195:3597]@code_fd.T # 최종수요를 6부문으로 만들기
fd.shape

In [ ]:
fd_tx = code_cns@icio_2[3195:3262,3195:3597]@code_fd.T
fd_tx.shape

In [ ]:
fd_row=np.concatenate((fd,fd_tx, np.zeros((2,18))))
fd_row.shape

In [ ]:
icio_row = np.concatenate((nae_tot_col,nae_tx, va_row))
icio_row.shape

In [ ]:
icio_col = np.concatenate((icio_row, fd_row), axis=1)

In [ ]:
icio_col_sm = icio_col.sum(axis=1).reshape([26,1]) # 충수요계(산출액과 동일)

In [ ]:
icio_f = np.concatenate((icio_col, icio_col_sm), axis=1).astype(float)

In [ ]:
ICIO_df = pd.DataFrame(icio_f, columns = [['KOR', 'KOR', 'KOR', 'CHN', 'CHN', 'CHN', 'USA', 'USA', 'USA', 'JPN', 'JPN','JPN',
                                             'EU', 'EU', 'EU', 'ROW', 'ROW', 'ROW',
                                            'KOR', 'KOR', 'KOR', 'CHN', 'CHN', 'CHN', 'USA', 'USA', 'USA', 'JPN', 'JPN','JPN',
                                             'EU', 'EU', 'EU', 'ROW', 'ROW', 'ROW', 'Total'],

                                      ['Agr', 'Manu', 'Serv', 'Agr', 'Manu', 'Serv','Agr', 'Manu', 'Serv', 'Agr', 'Manu', 'Serv','Agr', 'Manu', 'Serv','Agr', 'Manu', 'Serv',
                                        'Cons', 'Inv','Exp','Cons', 'Inv','Exp','Cons', 'Inv','Exp','Cons', 'Inv','Exp','Cons', 'Inv','Exp','Cons', 'Inv','Exp','Tot']],

                                    index=[['KOR', 'KOR', 'KOR', 'CHN', 'CHN', 'CHN', 'USA', 'USA', 'USA', 'JPN', 'JPN','JPN',
                                             'EU', 'EU', 'EU', 'ROW', 'ROW', 'ROW','Tax_KOR','Tax_CHN','Tax_USA','Tax_JPN','Tax_EU','Tax_ROW','VA','CT'],
                                           ['Agr', 'Manu', 'Serv', 'Agr', 'Manu', 'Serv','Agr', 'Manu', 'Serv', 'Agr', 'Manu', 'Serv',
                                            'Agr', 'Manu', 'Serv','Agr', 'Manu', 'Serv','Tax_KOR','Tax_CHN','Tax_USA','Tax_JPN','Tax_EU','Tax_ROW','VA','Output']])
ICIO_df.index.names=['Country', 'Secs']

In [ ]:
ICIO_df.to_csv('ICIO_6cons_3secs.csv')

In [ ]:
ICIO_df_show = pd.DataFrame(icio_f.round(1), columns = [['KOR', 'KOR', 'KOR', 'CHN', 'CHN', 'CHN', 'USA', 'USA', 'USA', 'JPN', 'JPN','JPN',
                                             'EU', 'EU', 'EU', 'ROW', 'ROW', 'ROW',
                                            'KOR', 'KOR', 'KOR', 'CHN', 'CHN', 'CHN', 'USA', 'USA', 'USA', 'JPN', 'JPN','JPN',
                                             'EU', 'EU', 'EU', 'ROW', 'ROW', 'ROW', 'Total'],

                                      ['Agr', 'Manu', 'Serv', 'Agr', 'Manu', 'Serv','Agr', 'Manu', 'Serv', 'Agr', 'Manu', 'Serv','Agr', 'Manu', 'Serv','Agr', 'Manu', 'Serv',
                                        'Cons', 'Inv','Exp','Cons', 'Inv','Exp','Cons', 'Inv','Exp','Cons', 'Inv','Exp','Cons', 'Inv','Exp','Cons', 'Inv','Exp','Tot']],

                                    index=[['KOR', 'KOR', 'KOR', 'CHN', 'CHN', 'CHN', 'USA', 'USA', 'USA', 'JPN', 'JPN','JPN',
                                             'EU', 'EU', 'EU', 'ROW', 'ROW', 'ROW','Tax_KOR','Tax_CHN','Tax_USA','Tax_JPN','Tax_EU','Tax_ROW','VA','CT'],
                                           ['Agr', 'Manu', 'Serv', 'Agr', 'Manu', 'Serv','Agr', 'Manu', 'Serv', 'Agr', 'Manu', 'Serv',
                                            'Agr', 'Manu', 'Serv','Agr', 'Manu', 'Serv','Tax_KOR','Tax_CHN','Tax_USA','Tax_JPN','Tax_EU','Tax_ROW','VA','Output']])
ICIO_df.index.names=['Country', 'Secs']

In [ ]:
ICIO_df_show